In [182]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import metrics as mt

In [183]:
# Datasets Path
TR_PATH = "./monks/datasets/monks-1.train"
TS_PATH = "./monks/datasets/monks-1.test"

In [184]:


import torch
import torch.nn as nn

class ThreeLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ThreeLayerNN, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        
        self.layer2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        
        self.layer3 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()



    def forward(self, x):
        
        x = self.layer1(x)
        x = self.relu1(x)

        x = self.layer2(x)
        x = self.relu2(x)

        x = self.layer3(x)
        x = self.sigmoid(x)

        return x


In [185]:
def read_ds(path):
  """
  parse CSV data set and
  returns a tuple (input, target)
  """
  df = pd.read_csv(path, sep=" ", names=['NaN','y','x1','x2','x3','x4','x5','x6','garbage'])
  y, df = df['y'], df.drop(columns=['NaN','garbage','y'])
  
  # One-hot encoding categorical variables
  df = pd.get_dummies(df, columns=['x1','x2','x3','x4','x5','x6']).astype('int')

  return (df, y)

In [186]:
# read training and test set
train_data, train_labels = read_ds(TR_PATH)
val_data,  val_labels  = read_ds(TS_PATH)

In [187]:
train_data = torch.tensor(train_data.values).float()
train_labels = torch.tensor(train_labels.values).float()

val_data = torch.tensor(val_data.values).float()
val_labels = torch.tensor(val_labels.values).float()



In [188]:
from itertools import product
import torch.optim as optim

hidden_sizes = [10, 20, 30]
learning_rates = [0.001, 0.01, 0.1]
epochs = 1000
input_size = len(val_data[0])
output_size = 1

best_params = None

for hidden_size, learning_rate in product(hidden_sizes, learning_rates):

    net = ThreeLayerNN(input_size, hidden_size, output_size)
    criterion = nn.BCELoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate)
    for epoch in range(epochs) :
        # Forward pass
        outputs = net(train_data).view(-1)
        # Compute the training loss
        loss = criterion(outputs, train_labels)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Evaluate on the validation set
    with torch.no_grad():
        val_outputs = net(val_data).view(-1)
        val_loss = criterion (val_outputs, val_labels)

    if best_params is None or val_loss < best_params[0]:
        best_params = (val_loss, hidden_size, learning_rate)

print(f"Best Hidden size: {hidden_size}, Learning rate: {learning_rate}, Validation loss: {val_loss}")



Best Hidden size: 30, Learning rate: 0.1, Validation loss: 0.012192058376967907
